# 📘 Text Generation-Lecture

> **주의:** Colab 런타임을 재시작하면 환경 변수가 초기화되므로 `OPENAI_API_KEY`를 다시 설정해야 합니다.


## 진행 순서
1. 환경 준비 및 API key 설정
2. GPT-5 계열 모델 개요 및 주요 파라미터
3. 모델별 출력 차이 비교 실습
4. Text Generation 개념과 instruction/input 구조
5. reasoning.effort · text.verbosity · max_output_tokens 제어
6. 멀티턴 대화 구성 실습 (스트리밍 응답 데모 포함)

## 1) 환경 준비 및 API 키 설정

In [ ]:
pip install openai

### API 키 설정 및 Client 생성
- Colab 메뉴 `보안 비밀(열쇠 아이콘)`에 `OPENAI_API_KEY`를 저장해 두었다면 자동으로 불러옵니다.
- 아래의 코드에서 오류가 난다면 API 키의 오류일 가능성이 높습니다.

In [ ]:
import os

OPENAI_API_KEY = None
try:
    from google.colab import userdata  # type: ignore
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except Exception:
    print("Colab 메뉴의 보안 비밀(열쇠 아이콘)에서 환경 변수가 제대로 저장되었는지 확인해주세요.")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
print("✅ API 키 설정 완료")

In [ ]:
from openai import OpenAI

client = OpenAI()
print("✅ 클라이언트 생성 완료")

### OpenAI API 시작하기
- Responses API를 사용하면 챗GPT처럼 프롬프트에 따라 다양한 텍스트(코드, 수식, JSON, 자연어 등)를 간편하게 만들 수 있습니다.

In [ ]:
response = client.responses.create(
    model="gpt-5-nano",
    instructions="You are a helpful assistant. Reply in Korean.",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

모델이 생성한 콘텐츠는 response.output 속성에서 확인할 수 있으며, 아래 예시는 출력(response) 형태를 보여줍니다.

In [ ]:
{
  "id": "resp_07354f378005a78400690d921c77408191b65a4eae80d15ac3",
  "created_at": 1762497052.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "You are a helpful assistant. Reply in Korean.",
  "metadata": {},
  "model": "gpt-5-nano-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "rs_07354f378005a78400690d921d59b48191baa7723fcf20871b",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "msg_07354f378005a78400690d9223f82081918b05a37be09e6764",
      "content": [
        {
          "annotations": [],
          "text": "밤하늘 아래 작은 숲에서 한 마리의 유니콘이 부드러운 빛을 내뿜으며 아이의 꿈으로 길을 이끌었고, 아이는 포근한 잠에 들었다.",
          "type": "output_text",
          "logprobs": []
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [],
  "top_p": 1.0,
  "background": false,
  "conversation": null,
  "max_output_tokens": null,
  "max_tool_calls": null,
  "previous_response_id": null,
  "prompt": null,
  "prompt_cache_key": null,
  "reasoning": {
    "effort": "medium",
    "generate_summary": null,
    "summary": null
  },
  "safety_identifier": null,
  "service_tier": "default",
  "status": "completed",
  "text": {
    "format": {
      "type": "text"
    },
    "verbosity": "medium"
  },
  "top_logprobs": 0,
  "truncation": "disabled",
  "usage": {
    "input_tokens": 31,
    "input_tokens_details": {
      "cached_tokens": 0
    },
    "output_tokens": 950,
    "output_tokens_details": {
      "reasoning_tokens": 896
    },
    "total_tokens": 981
  },
  "user": null,
  "billing": {
    "payer": "developer"
  },
  "prompt_cache_retention": null,
  "store": true
}

In [ ]:
# 내가 그냥 한번 해보려고 넣은거
output = response.output
print(output)

모델이 생성한 콘텐츠 배열은 `response.output` 속성에 담기며, 아래와 같은 형태입니다.

이 배열에는 도구 호출, 추론 모델이 만든 reasoning 토큰 데이터 등 여러 항목이 함께 들어갈 수 있습니다. 

In [ ]:
"output": [
    {
      "id": "rs_07354f378005a78400690d921d59b48191baa7723fcf20871b",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "msg_07354f378005a78400690d9223f82081918b05a37be09e6764",
      "content": [
        {
          "annotations": [],
          "text": "밤하늘 아래 작은 숲에서 한 마리의 유니콘이 부드러운 빛을 내뿜으며 아이의 꿈으로 길을 이끌었고, 아이는 포근한 잠에 들었다.",
          "type": "output_text",
          "logprobs": []
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ]

## 2) 모델 선택 및 주요 파라미터

### model
- 응답을 생성할 모델 ID(`gpt-5`, `o3` 등)을 지정합니다.
- 모델별 성능과 비용은 [OpenAI 모델 가이드](https://platform.openai.com/docs/models)에서 최신 정보를 확인하세요.

### instructions
- 모델 컨텍스트에 넣는 시스템/개발자 지침으로, 응답의 톤·역할·제약을 정의합니다.
- `previous_response_id`와 함께 사용하면 이전 턴 instructions를 끊고 새 instruction으로 쉽게 교체할 수 있습니다.
    - `previous_response_id`: 이전 응답의 고유 ID입니다. 이 값을 활용하면 멀티턴 대화를 구성할 수 있습니다.

### input
- 텍스트, 이미지, 파일 등 모델이 답변을 생성할 때 참고하는 입력값입니다.

### max_output_tokens
- 가시 출력 토큰(visible output token)과 추론 토큰(resoning token)을 모두 포함한 최대 생성 길이를 설정합니다.

> 프롬프트 엔지니어링
프롬프트 엔지니어링은 모델에 효과적인 지시문을 작성하여 요구사항에 맞는 결과를 일관되게 생성하도록 하는 과정입니다.
모델이 생성하는 결과는 비결정적이므로, 원하는 출력을 얻기 위한 프롬프팅은 예술과 과학의 조합입니다. 그럼에도 기술과 모범 사례를 적용하면 일관되게 좋은 결과를 얻을 수 있습니다.
메시지 역할 사용과 같은 일부 프롬프트 엔지니어링 기법은 모든 모델에서 작동합니다. 하지만 모델마다 최상의 결과를 얻기 위해 다른 프롬프팅이 필요할 수 있습니다. 같은 계열 모델의 서로 다른 스냅샷도 다른 결과를 생성할 수 있습니다. 따라서 더 복잡한 애플리케이션을 구축할 때는 다음을 권장합니다:
프로덕션 애플리케이션을 특정 모델 스냅샷(예: gpt-5-2025-08-07)에 고정하여 일관된 동작 보장
프롬프트의 동작을 측정하는 평가를 구축하여 반복하거나 모델 버전 변경/업그레이드 시 프롬프트 성능 모니터링
프롬프트를 구성하는 데 사용할 수 있는 도구와 기법을 살펴보겠습니다.

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
        model="gpt-5-mini",
        instructions="You are a helpful assistant. Reply in Korean.", 
        input="Write a one-sentence bedtime story about a unicorn.",
    )

print(response.output_text)

- `instructions` 파라미터는 응답 생성 시 우선 적용되는 전역 지시를 제공합니다.
- `instructions`는 톤, 목표, 올바른 응답 예시 등 행동 지침을 정의하며 `input` 파라미터보다 높은 우선순위를 가집니다.

> instruction 수정: 해적처럼 말해줘.\
instruction 수정: 일본어로 답해줘.\
input 수정: write a three-sentence bedtime story about a rabbit

In [ ]:
from openai import OpenAI

client = OpenAI()

# max_output_tokens 예제
for tokens in [300, 500, 1000]:
    response = client.responses.create(
        model="gpt-5-mini",
        instructions="You are a helpful assistant. Reply in Korean.",
        input="Write a one-sentence bedtime story about a unicorn.",
        max_output_tokens=tokens
    )
    print(f"\n--- max_output_tokens={tokens} ---")
    print(response.output_text)

### GPT-5 모델 라인업

GPT-5 시리즈는 총 세 가지 모델이 있습니다. 선택 시 아래 특성을 참고하세요.

- gpt-5: 복잡한 추론, 광범위한 상식, 코드 작성 또는 다단계 작업 등 가장 어려운 문제에 적합
- gpt-5-mini: 합리적인 비용에 균형 잡힌 성능과 속도를 제공하며, 채팅이나 일반적 과제에 추천
- gpt-5-nano: 아주 빠른 처리와 저렴한 비용이 장점이며, 단순 분류·지시 수행 등 대량 처리 용도에 적합

일반적으로 모델이 작아질수록 비용과 대기시간은 낮아지지만, 범용 상식이나 창의성이 다소 낮을 수 있습니다.
모든 작업에 큰 모델이 필요한 것은 아닙니다. 잘 정의된 과제나 구조화된 요청의 경우, 작은 모델이 오히려 더 효율적이고 경제적일 수 있습니다.

그 외 더 많은 모델은 아래 링크에서 확인해주세요.
https://platform.openai.com/docs/models

> **주의**: GPT-5 계열은 `temperature`, `top_p`, `logprobs` 파라미터를 지원하지 않습니다. 대신 `reasoning.effort`, `text.verbosity`, `max_output_tokens`로 제어하세요.

## 3) 모델별 출력 차이 비교 실습

In [ ]:
from openai import OpenAI

client = OpenAI()

llm_prompt = "대형 언어 모델(LLM)이 무엇인지 설명하세요."
response = client.responses.create(
    model="gpt-5-nano", # gpt-5, gpt-5-mini, gpt-5-nano
    instructions="You are a helpful instructor. Reply in Korean and keep the answer well-structured.",
    input=llm_prompt,
)
print(response.output_text)

> GPT-5 이외에도 옛날 모델들도 한번 돌려보도록 하기, 4o나 3.5는 어떤 얘기를 하는지 한번 해보는거지 

내가 볼려고 넣은 개념

✅ 스트리밍 응답 처리 흐름 정리
 
GPT-5 API에서 스트림 응답을 받을 때 도착하는 이벤트와 처리 순서 예시입니다.
| 단계 | event.type                    | 설명                             | 예시 출력                        |
|------|-------------------------------|----------------------------------|----------------------------------|
| 1    | response.output_text.delta    | 모델이 생성한 텍스트 일부 도착    | "대형", " 언어", " 모델은" ...   |
| 2    | response.refusal.delta        | (선택적) 모델이 거절 메시지 전송  | "죄송합니다, 해당 요청은..."     |
| 3    | response.error                | 오류 발생                        | "Rate limit exceeded"            |
| 4    | response.completed            | 모든 토큰 생성 종료 알림          | "Completed"                      |
| 5    | stream.get_final_response()   | 전체 응답 결과(최종 텍스트 등)    | (최종 생성 문장 전체 반환)       |
- 보통 단계 1에서 여러 번 event가 들어오며, 각 토큰이나 조각마다 도착합니다.
- 단계 5는 모든 스트리밍 종료 후 한 번만 호출하여, 통합 결과를 얻을 수 있습니다.
- 거절(event.type = response.refusal.delta) 또는 오류(event.type = response.error) 발생 시에는 이후 생성이 중단될 수 있습니다.
- 실 서비스에서는 각 event.type에 따라 출력 처리, 예외처리를 분기하세요.

In [ ]:
from openai import OpenAI

client = OpenAI()

# 이건 좀 하면서, 너무 오래걸리니, stream 기능을 한번 해보겠다는 식으로 전개하면 좋겠다.
# 스트림을 어떻게 사용하는지 보고 싶으시면, 독스를 보시라
llm_prompt = "대형 언어 모델(LLM)이 무엇인지 설명하세요."
resp = client.responses.create(
    model="gpt-5-nano", # gpt-5, gpt-5-mini, gpt-5-nano
    instructions="You are a helpful instructor. Reply in Korean and keep the answer well-structured.",
    input=llm_prompt,
    stream = True,
)

for event in resp:
    print(event)

# 위와 같이 하면 안되고..

for event in resp:
    if event.type == "response.output_text.delta":
        print(event.delta, end="")
        

#print(resp.output_text)

## 5) reasoning.effort · text.verbosity · max_output_tokens 제어

### reasoning.effort 파라미터는 모델이 응답을 생성하기 전 얼마나 많은 추론 토큰을 사용할지 결정합니다.
- reasoning effort는 minimal, low, medium, high를 지원합니다.
   - minimal: 빠르고, 최대한 간결하게 답변
   - low: 빠른 속도, 토큰 적게 사용
   - medium: 속도-정확성 균형, GPT-5의 기본값
   - high: 꼼꼼하고, 깊이 있는 설명 제공


In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-5-mini",
    input="남산타워 외벽을 1mm 두께의 금으로 도금하려면 금이 몇 kg이나 필요할까요?",
    reasoning={
        "effort": "minimal" # low, medium, high
    }
)

print(response)

### text.verbosity 파라미터는 출력 답변의 길이 (토큰 수)에 영향을 주는 설정입니다.
- verbosity는 low, medium, high 옵션을 지원합니다.
   - low: 짧고, 간결하며 요점만 전달 (예: 요약, 코드 스니펫)
   - medium: 적절한 상세 설명과 길이, GPT-5의 기본값
   - high: 길고, 자세하며 세부 설명이 많은 답변 제공
- 응용 목적에 따라 상세한 설명이 필요하면 high, 아주 짧은 결과가 요구되면 low를 사용할 수 있습니다.
- 실제 답변 길이는 입력 프롬프트, max_output_tokens 등 다른 설정에도 영향받습니다.

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-5-mini",
    input="삶, 우주 그리고 모든 것에 대한 궁극적인 질문의 답은 무엇인가요?",
    text={
        "verbosity": "low" # medium, high
    }
)

print(response)

> 실습 지시: 원하는 파라미터 조합만 남기고 싶다면 루프를 주석 처리하거나 선택적으로 실행하도록 수정해 보세요.

### 메시지 역할(role)
| 역할(role)     | 설명                                                    | 우선순위                                 |
| -------------- | ------------------------------------------------------- | ---------------------------------------- |
| developer      | 애플리케이션 개발자가 제공하는 instructions             | 가장 높음 (user 메시지보다 우선)         |
| user           | 사용자가 제공하는 instructions                        | developer 메시지보다 낮고 assistant보다 높음 |
| assistant      | 모델이 생성하는 응답 메시지                              | 가장 낮음                                |

- OpenAI 모델 스펙에서는 역할 우선순위를 `developer` > `user` > `assistant` 순으로 정의합니다.

### 메시지 역할을 사용한 동일 예시

### 💡 왜 input을 리스트 형태로 사용할까요?

위 예시에서 `input`을 리스트 형태로 사용한 이유를 이해하는 것이 중요합니다:

1. **역할(role) 구분의 필요성**
   - `role: "developer"`: 시스템 지시 (항상 최우선)
   - `role: "user"`: 사용자 입력
   - `role: "assistant"`: 모델의 응답 (멀티턴 대화에서 사용)

2. **멀티턴 대화의 필수 구조**
   - 대화 히스토리를 유지하려면 이전 메시지들을 모두 포함해야 함
   - 각 메시지의 역할을 명시해야 모델이 올바르게 해석함
   - 예: `[developer 메시지, user 질문1, assistant 답변1, user 질문2]` 형태

3. **instructions 파라미터와의 관계**
   - `instructions` 파라미터는 단일 요청에 적합
   - 멀티턴 대화에서는 `input` 리스트 내 `role: "developer"`가 더 유연함
   - 둘 다 같은 목적(시스템 지시)이지만, 사용 상황에 따라 선택

**핵심**: 단순한 질문은 `instructions` + 문자열 `input`으로 충분하지만, 대화를 이어가려면 리스트 형태의 `input`이 더 적합합니다.

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-5-mini",
    reasoning={"effort": "low"},
    input=[
        {
            "role": "developer",
            "content": "해적처럼 말해 주세요."
        },
        {
            "role": "user",
            "content": "한국의 수도는 부산인가요?"
        }
    ]
)

print(response.output_text)

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-5-mini",
    input=[
        {"role": "user", "content": "knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
)

print(response.output_text)

강의에서 위 내용 설명할 때, 말씀드리면 될 것 같다. 

By using alternating user and assistant messages, you capture the previous state of a conversation in one request to the model.

To manually share context across generated responses, include the model's previous response output as input, and append that input to your next request.

In the following example, we ask the model to tell a joke, followed by a request for another joke. Appending previous responses to new requests in this way helps ensure conversations feel natural and retain the context of previous interactions.

## 6) 멀티턴 대화 구성 실습

### 멀티턴 컨텍스트 관리 기본 개념
- Responses API는 자동으로 대화 히스토리를 보존하지 않으므로 애플리케이션에서 직접 메시지를 누적해야 합니다.
- 각 턴마다 `role`과 `content`가 있는 딕셔너리를 추가하고, 모델이 생성한 응답도 히스토리에 포함해야 다음 요청에서 참조할 수 있습니다.
- `previous_response_id`를 사용하면 이전 응답의 입력과 출력을 간단히 재사용해 체인 또는 분기형 기록을 만들 수 있습니다.

In [ ]:
response = client.responses.create(
    model="gpt-5-mini",
    instructions="You are a helpful assistant. You must answer in Korean.",
    input="대한민국의 수도는 어디인가요?",
)

print(response.output_text)

In [ ]:
def ask(question):
    response = client.responses.create(
        model="gpt-5-mini",
        instructions="You are a helpful assistant. You must answer in Korean.",
        input=question,
    )
    return response.output_text

In [ ]:
# 첫 번째 질문
ask("대한민국의 수도는 어디인가요?")

In [ ]:
# 두 번째 질문
ask("방금 답변을 영어로 번역해 주세요")

위의 예시처럼 API를 매번 단일 호출로만 사용하면 모델이 **이전 대화를 기억하지 못해 엉뚱한 답변**을 할 수 있습니다. 이는 대화 기록을 저장하지 않았기 때문입니다.

해결 방법은 간단합니다. 각 턴의 사용자 질문과 모델 응답을 순서대로 히스토리에 추가해 다음 요청의 `input`으로 전달하면, 챗봇이 이전 문맥을 자연스럽게 참고할 수 있습니다.

In [ ]:
context = [
    {
        "role": "developer",
        "content": "You are a helpful assistant. You must answer in Korean.",
    },
    {
        "role": "user",
        "content": "대한민국의 수도는 어디인가요?",
    },
]

res1 = client.responses.create(
    model="gpt-5-mini",
    input=context,
)

first_answer = res1.output_text
print("첫 번째 응답:", first_answer)

In [ ]:
context.append({"role": "assistant", "content": first_answer})
context.append([{"role": el.role, "content": el.content} for el in res1.output])
context.append({"role": "user", "content": "방금 답변을 영어로 번역해 주세요."})

res2 = client.responses.create(
    model="gpt-5-mini",
    input=context,
)

print("\n두 번째 응답:", res2.output_text)

이제 Responses API 호출을 함수로 감싸서 히스토리를 자동으로 관리해 보겠습니다.

In [ ]:
def ask(question, message_history=None, *, model="gpt-5-mini", reasoning_effort="medium", verbosity="medium"):
    if message_history is None:
        message_history = [
            {
                "role": "developer",
                "content": "You are a helpful assistant. You must answer in Korean.",
            }
        ]

    message_history.append({"role": "user", "content": question})

    response = client.responses.create(
        model=model,
        input=message_history,
        reasoning={"effort": reasoning_effort},
        text={"verbosity": verbosity},
    )

    message_history.append({"role": "assistant", "content": response.output_text})
    return message_history

In [ ]:
# 최초 질문
message_history = ask("양자역학에 대해서 쉽게 설명해 주세요")
# 최초 답변
print(message_history[-1])

In [ ]:
message_history

In [ ]:
# 두 번째 질문
message_history = ask(
    "이전의 내용을 영어로 답변해 주세요", message_history=message_history
)
# 두 번째 답변
print(message_history[-1])

### previous_response_id를 활용한 멀티턴 대화 예시
- The unique ID of the previous response to the model.
- Use this to create multi-turn conversations. 

In [ ]:
context = [
    {"role": "developer", "content": "You are a helpful assistant. You must answer in Korean."},
    {"role": "user", "content": "재밌는 농담 해줘."},
]

res1 = client.responses.create(
    model="gpt-5-mini",
    input=context,
)

print(res1.output_text)

In [ ]:
# 이전 응답의 response_id를 활용하여 대화 이어가기
res2 = client.responses.create(
    model="gpt-5-mini",
    input=[{"role": "user", "content": "그게 왜 재밌는건지 설명해줘."}],
    previous_response_id=res1.id,
)

print(res2.output_text)

Another way to manage conversation state is to share context across generated responses with the previous_response_id parameter. This parameter lets you chain responses and create a threaded conversation.
대쉬보드에 가서도 응답에 대해 확인해볼 수 있음
30일 간 기본적으로 보관되며, 끄고 싶으면, store=false 옵션을 넣어서 끌 수 있다

### 참고 자료
- https://platform.openai.com/docs
